# Another attempt at MC Simulation on AHP/ANP

The ideas are the following:

1. There is a class MCAnp that has a sim() method that will simulate any Prioritizer
2. MCAnp also has a sim_fill() function that does fills in the data needed for a single simulation

## Import needed libs

In [1]:
import pandas as pd
import sys 
import os
sys.path.insert(0, os.path.abspath("../"))
import numpy as np
from scipy.stats import triang
from copy import deepcopy
from pyanp.priority import pri_eigen
from pyanp.pairwise import Pairwise
from pyanp.ahptree import AHPTree, AHPTreeNode
from pyanp.direct import Direct

# MCAnp class

In [2]:
def ascale_mscale(val:(float,int))->float:
    if val is None:
        return 0
    elif val < 0:
        val = -val
        val += 1
        val = 1.0/val
        return val
    else:
        return val+1
    
def mscale_ascale(val:(float,int))->float:
    if val == 0:
        return None
    elif val >= 1:
        return val - 1
    else:
        val = 1/val
        val = val-1
        return -val

In [87]:
DEFAULT_DISTRIB = triang(c=0.5, loc=-1.5, scale=3.0)
def avote_random(avote):
    """
    Returns a random additive vote in the neighborhood of the additive vote avote
    according to the default disribution DEFAULT_DISTRIB
    """
    if avote is None:
        return None
    raw_val = DEFAULT_DISTRIB.rvs(size=1)[0]
    return avote+raw_val


def mvote_random(mvote):
    """
    Returns a random multiplicative vote in the neighborhhod of the multiplicative vote mvote
    according to the default distribution DEFAULT_DISTRIB.  This is handled by converting
    the multiplicative vote to an additive vote, calling avote_random() and converting the
    result back to an additive vote
    """
    avote = mscale_ascale(mvote)
    rval_a = avote_random(avote)
    rval = ascale_mscale(rval_a)
    return rval

def direct_random(direct, max_percent_chg=0.2)->float:
    """
    Returns a random direct data value near the value `direct'.  This function
    creates a random percent change, between -max_percent_chg and +max_percent_chg, and
    then changes the direct value by that factor, and returns it.
    """
    pchg = np.random.uniform(low=-max_percent_chg, high=max_percent_chg)
    return direct * (1 + pchg)
    
class MCAnp:
    def __init__(self):
        # Setup the random pairwise vote generator
        self.pwvote_random = mvote_random
        # Setup the random direct vote generator
        self.directvote_random = direct_random
        # Set the default user to use across the simulation
        # follows the standard from Pairwise class, i.e. it can be a list
        # of usernames, a single username, or None (which means total group average)
        self.username = None
        # What is the pairwise priority calculation?
        self.pwprioritycalc = pri_eigen
        
    def sim_fill(self, src, dest):
        """
        Fills in data on a structure prior to doing the simulation calculations.
        This function calls sim_NAME_fill depending on the class of the src object.
        If the dest object is None, we create a dest object by calling deepcopy().
        In either case, we always return the allocated dest object
        """
        if dest is None:
            dest = deepcopy(src)
        # Which kind of src do we have
        if isinstance(src, np.ndarray):
            # We are simulating on a pairwise comparison matrix
            return self.sim_pwmat_fill(src, dest)
        elif isinstance(src, Pairwise):
            # We are simulating on a multi-user pairwise comparison object
            return self.sim_pw_fill(src, dest)
        elif isinstance(src, AHPTree):
            # We are simulating on an ahp tree object
            return self.sim_ahptree_fill(src, dest)
        elif isinstance(src, Direct):
            # We are simulating on an ahp direct data
            return self.sim_direct_fill(src, dest)
        else:
            raise ValueError("Src class is not handled, it is "+type(src).__name__)
    
    def sim_pwmat_fill(self, pwsrc:np.ndarray, pwdest:np.ndarray=None)->np.ndarray:
        """
        Fills in a pairwise comparison matrix with noisy votes based on pwsrc
        If pwsrc is None, we create a new matrix, otherwise we fill in pwdest
        with noisy values based on pwsrc and the self.pwvote_random parameter.
        In either case, we return the resulting noisy matrix
        """
        if pwdest is None:
            pwdest = deepcopy(pwsrc)
        size = len(pwsrc)
        for row in range(size):
            pwdest[row,row] = 1.0
            for col in range(row+1, size):
                val = pwsrc[row,col]
                if val >= 1:
                    nvote = self.pwvote_random(val)
                    pwdest[row, col]=nvote
                    pwdest[col, row]=1/nvote
                elif val!= 0:
                    nvote = self.pwvote_random(1/val)
                    pwdest[col, row] = nvote
                    pwdest[row, col] = 1/nvote
                else:
                    pwdest[row, col] = nvote
                    pwdest[col, row] = nvote
        return pwdest
    
    def sim_pwmat(self, pwsrc:np.ndarray, pwdest:np.ndarray=None)->np.ndarray:
        """
        creates a noisy pw comparison matrix from pwsrc, stores the matrix in pwdest (which
        is created if pwdest is None) calculates the resulting priority and returns that
        """
        pwdest = self.sim_pwmat_fill(pwsrc, pwdest)
        rval = self.pwprioritycalc(pwdest)
        return rval
    
    def sim_pw(self, pwsrc:Pairwise, pwdest:Pairwise)->np.ndarray:
        """
        Performs a simulation on a pairwise comparison matrix object and returns the
        resulting priorities
        """
        pwdest = self.sim_pw_fill(pwsrc, pwdest)
        mat = pwdest.matrix(self.username)
        rval = self.pwprioritycalc(mat)
        return rval
        
    
    def sim_pw_fill(self, pwsrc:Pairwise, pwdest:Pairwise=None)->Pairwise:
        """
        Fills in the pairwise comparison structure of pwdest with noisy pairwise data from pwsrc.
        If pwdest is None, we create one first, then fill in.  In either case, we return the pwdest
        object with new noisy data in it.
        """
        if pwdest is None:
            pwdest = deepcopy(pwsrc)
        for user in pwsrc.usernames():
            srcmat = pwsrc.matrix(user)
            destmat = pwdest.matrix(user)
            self.sim_pwmat_fill(srcmat, destmat)
        return pwdest
    
    def sim_direct_fill(self, directsrc:Direct, directdest:Direct=None)->Direct:
        """
        Fills in the direct data structure of directdest with noisy data from directsrc.
        If directdest is None, we create on as a deep copy of directsrc, then fill in.
        In either case, we return the directdest object with new noisy data in it.
        """
        if directdest is None:
            directdest = deepcopy(directsrc)
        for altpos in range(len(directdest)):
            orig = directsrc[altpos]
            newvote = self.directvote_random(orig)
            directdest.data[altpos] = newvote
        return directdest
        
    def sim_direct(self, directsrc:Direct, directdest:Direct=None)->np.ndarray:
        """
        Simulates for direct data
        """
        directdest = self.sim_direct_fill(directsrc, directdest)
        return directdest.priority()
    
    def sim_ahptree_fill(self, ahpsrc:AHPTree, ahpdest:AHPTree)->AHPTree:
        """
        Fills in the ahp tree structure of ahpdest with noisy data from ahpsrc.
        If ahpdest is None, we create one as a deepcopy of ahpsrc, then fill in.
        In either case, we return the ahpdest object with new noisy data in it.
        """
        if ahpdest is None:
            ahpdest = deepcopy(ahpsrc)
        self.sim_ahptreenode_fill(ahpsrc.root, ahpdest.root)
        return ahpdest
    
    def sim_ahptreenode_fill(self, nodesrc:AHPTreeNode, nodedest:AHPTreeNode)->AHPTreeNode:
        """
        Fills in data in an AHPTree
        """
        #Okay, first we fill in for the alt_prioritizer
        if nodesrc.alt_prioritizer is not None:
            self.sim_fill(nodesrc.alt_prioritizer, nodedest.alt_prioritizer)
        #Now wefill in the child prioritizer
        if nodesrc.child_prioritizer is not None:
            self.sim_fill(nodesrc.child_prioritizer, nodedest.child_prioritizer)
        #Now for each child, fill in
        for childsrc, childdest in zip(nodesrc.children, nodedest.children):
            self.sim_ahptreenode_fill(childsrc, childdest)
        #We are done, return the dest
        return nodedest
    
    def sim_ahptree(self, ahpsrc:AHPTree, ahpdest:AHPTree)->np.ndarray:
        """
        Perform the actual simulation
        """
        ahpdest = self.sim_ahptree_fill(ahpsrc, ahpdest)
        return ahpdest.priority()

In [88]:
mc = MCAnp()

In [89]:
pw = np.array([
    [1, 1/2, 3],
    [2, 1, 5],
    [1/3, 1/5, 1]
])
rpw= mc.sim_pwmat_fill(pw)
rpw

array([[1.        , 0.54012281, 2.23788435],
       [1.85143078, 1.        , 4.82623154],
       [0.44685062, 0.207201  , 1.        ]])

In [90]:
[mc.sim_pwmat(pw) for i in range(20)]

[array([0.26249346, 0.63754787, 0.09995867]),
 array([0.31198057, 0.58650765, 0.10151178]),
 array([0.37505849, 0.51450444, 0.11043707]),
 array([0.3292281 , 0.56572839, 0.10504351]),
 array([0.3483477 , 0.53085788, 0.12079442]),
 array([0.26655757, 0.62765525, 0.10578719]),
 array([0.28788247, 0.59649657, 0.11562096]),
 array([0.2472799 , 0.66451076, 0.08820934]),
 array([0.250429  , 0.63557177, 0.11399923]),
 array([0.4589622 , 0.43678513, 0.10425268]),
 array([0.30966134, 0.58805155, 0.10228711]),
 array([0.29917877, 0.59350961, 0.10731162]),
 array([0.30957021, 0.58310374, 0.10732605]),
 array([0.34758191, 0.52954871, 0.12286938]),
 array([0.3309997, 0.5610149, 0.1079854]),
 array([0.27348453, 0.62040941, 0.10610606]),
 array([0.323606  , 0.56773161, 0.10866239]),
 array([0.37073211, 0.52072318, 0.10854471]),
 array([0.33180072, 0.54282838, 0.1253709 ]),
 array([0.35241972, 0.54535137, 0.10222891])]

In [91]:
pwobj = Pairwise(alts=['alt '+str(i) for i in range(3)])
pwobj.vote_matrix(user_name='u1', val=pw)

## Checking that the deep copy is actually a deep copy
For some reason deepcopy was not copying the matrix, I had to overwrite
__deepcopy__ in Pairwise

In [92]:
pwobj.matrix('u1')

array([[1.        , 0.5       , 3.        ],
       [2.        , 1.        , 5.        ],
       [0.33333333, 0.2       , 1.        ]])

In [93]:
rpwobj = pwobj.__deepcopy__()

In [94]:
a=rpwobj
b=pwobj
a.df

,Name,Age,Matrix
u1,None,None,"[[1.0, 0.5, 3.0], [2.0, 1.0, 5.0], [0.33333333..."


In [95]:
display(a.df.loc['u1', 'Matrix']) 
display(b.df.loc['u1', 'Matrix'])

array([[1.        , 0.5       , 3.        ],
       [2.        , 1.        , 5.        ],
       [0.33333333, 0.2       , 1.        ]])

array([[1.        , 0.5       , 3.        ],
       [2.        , 1.        , 5.        ],
       [0.33333333, 0.2       , 1.        ]])

In [96]:
display(a.matrix('u1') is b.matrix('u1'))
display(a.matrix('u1') == b.matrix('u1'))

False

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

## Now let's try to simulate

In [97]:
[mc.sim_pw(pwobj, rpwobj) for i in range(20)]

[array([0.3251892 , 0.56276057, 0.11205023]),
 array([0.41421788, 0.49254641, 0.09323571]),
 array([0.31487743, 0.55204376, 0.13307881]),
 array([0.28670449, 0.61124979, 0.10204571]),
 array([0.26998622, 0.60994123, 0.12007255]),
 array([0.31938507, 0.56124654, 0.11936839]),
 array([0.35447313, 0.51837345, 0.12715341]),
 array([0.41775525, 0.48012645, 0.1021183 ]),
 array([0.4302707 , 0.42907299, 0.14065631]),
 array([0.29128787, 0.60630827, 0.10240385]),
 array([0.27895485, 0.62133799, 0.09970717]),
 array([0.31514403, 0.54466222, 0.14019375]),
 array([0.31087064, 0.58334495, 0.10578441]),
 array([0.38209868, 0.50084884, 0.11705248]),
 array([0.30181485, 0.59125145, 0.1069337 ]),
 array([0.29521383, 0.59334821, 0.11143796]),
 array([0.38398711, 0.50812623, 0.10788667]),
 array([0.37948772, 0.49619075, 0.12432153]),
 array([0.26183954, 0.60592893, 0.13223153]),
 array([0.44653312, 0.43223428, 0.1212326 ])]

In [98]:
pwobj.matrix('u1')

array([[1.        , 0.5       , 3.        ],
       [2.        , 1.        , 5.        ],
       [0.33333333, 0.2       , 1.        ]])

## Try to simulate a direct data

In [99]:
dd = Direct(alt_names=['a1', 'a2', 'a3'])
dd.data[0]=0.5
dd.data[1]=0.3
dd.data[2]=0.2

In [100]:
rdd=mc.sim_direct_fill(dd)
rdd.data

a1    0.519515
a2    0.299819
a3    0.217881
dtype: float64

## Simulate an ahptree

In [101]:
alts=['alt '+str(i) for i in range(3)]
tree = AHPTree(alt_names=alts)
kids = ['crit '+str(i) for i in range(4)]
for kid in kids:
    tree.add_child(kid)
    node  = tree.get_node(kid)
    direct = node.alt_prioritizer
    s = 0
    for alt in alts:
        direct[alt] = np.random.uniform()
        s += direct[alt]
    if s != 0:
        for alt in alts:
            direct[alt] /= s
            

In [102]:
tree.priority()

alt 0    0.403956
alt 1    0.293827
alt 2    0.302217
dtype: float64

In [109]:
mc.sim_ahptree(tree, None)

alt 0    0.417154
alt 1    0.327032
alt 2    0.320193
dtype: float64

In [104]:
tree.priority()

alt 0    0.403956
alt 1    0.293827
alt 2    0.302217
dtype: float64